# Event Matching and Match Evaluation

In [ ]:
import os

import numpy as np
import pandas as pd

import peyes

## Preparing the Data
As shown in previous notebooks, we start by preparing the data.  
In the following code blocks we:  
1. Download the "Lund2013" dataset.  
2. Extract the data of the first trial, including the time, x, and y columns as well as the pixel size and viewer distance.  
3. Extract the annotations by the two human annotators _"RA"_ and _"MN"_. These will be used as the ground truth for the upcoming evaluation.  
4. Create two detector objects, using Engbert's detection algorithm and the I-VT detection algorithm.  
5. Use each detector to label the samples in the first trial.  

In [ ]:
# Download the "Lund2013" dataset
dataset = peyes.datasets.lund2013(directory=None, save=False, verbose=True)

# Extract the data of the first trial
trial1_data = dataset[dataset[peyes.constants.TRIAL_ID_STR] == 1]

# read labels from the two annotators
ra = trial1_data["RA"].values
mn = trial1_data["MN"].values

# Extract the time, x, and y columns as well as the pixel size and viewer distance
trial1_t = trial1_data[peyes.constants.T].values
trial1_x = trial1_data[peyes.constants.X].values
trial1_y = trial1_data[peyes.constants.Y].values
trial1_p = trial1_data[peyes.constants.PUPIL].values
trial1_pixel_size = trial1_data["pixel_size"].values[0]
trial1_viewer_distance = trial1_data["viewer_distance"].values[0]

# create detector objects and use them to detect the samples in the first trial
engbert = peyes.create_detector("engbert", missing_value=np.nan, min_event_duration=4, pad_blinks_time=0)
eng_labels, eng_metadata = engbert.detect(
    t=trial1_t, x=trial1_x, y=trial1_y, pixel_size_cm=trial1_pixel_size, viewer_distance_cm=trial1_viewer_distance
)

ivt = peyes.create_detector("ivt", missing_value=np.nan, min_event_duration=4, pad_blinks_time=0)
ivt_labels, ivt_metadata = ivt.detect(
    t=trial1_t, x=trial1_x, y=trial1_y, pixel_size_cm=trial1_pixel_size, viewer_distance_cm=trial1_viewer_distance
)

### Generating Events
As shown in notebook #1, consecutive samples that share the same label can be grouped into an `Event` object.  
These are one of `pEYES`'s data structures, and they contain many useful attributes and methods that are easier to use than manipulating the raw data directly. For example, we can use an `Event` object to calculate the duration of the event, instead of manually calculating the difference between the start and end times of the samples that compose the event.  

To convert `pEYES`'s labels to events, we use the `peyes.create_events()` function (see notebook #1 for more details).  
As another preparatory step, we will convert the labels from all annotators and detectors to their corresponding events.

In [ ]:
ra_events = peyes.create_events(
    labels=ra,
    t=trial1_t, x=trial1_x, y=trial1_y, pupil=trial1_p,
    pixel_size=trial1_pixel_size, viewer_distance=trial1_viewer_distance
)
mn_events = peyes.create_events(
    labels=mn,
    t=trial1_t, x=trial1_x, y=trial1_y, pupil=trial1_p,
    pixel_size=trial1_pixel_size, viewer_distance=trial1_viewer_distance
)
eng_events = peyes.create_events(
    labels=eng_labels,
    t=trial1_t, x=trial1_x, y=trial1_y, pupil=trial1_p,
    pixel_size=trial1_pixel_size, viewer_distance=trial1_viewer_distance
)
ivt_events = peyes.create_events(
    labels=ivt_labels,
    t=trial1_t, x=trial1_x, y=trial1_y, pupil=trial1_p,
    pixel_size=trial1_pixel_size, viewer_distance=trial1_viewer_distance
)